# Promote to Library Major Version

Merges ALL active DTA Major versions into a new Library Major version.

**Operation:** `PROMOTE_TO_LIBRARY`

**What it does:**
1. Queries `md_version_registry` for all active DTA_MAJOR versions
2. Reads records from Gold library for those DTA versions
3. Merges them using the specified strategy (dedup by definition_hash)
4. Creates new LIBRARY_MAJOR version (e.g., "2.0")
5. Closes previous LIBRARY_MAJOR using SCD Type 2

**Parameters:**
- `merge_strategy` - How to handle duplicates: UNION_DEDUP (default), LATEST_WINS, FIRST_WINS
- `library_type` - Library type (e.g., transfer_variables)

**Result:**
New library major version with merged definitions from all approved DTAs.


In [ ]:
# Cell 1: Imports
import json
from clinical_data_standards_framework.versioning import (
    merge_dtas_to_library_major,
    get_current_library_version,
    get_all_versions
)

print("="*80)
print("PROMOTE TO LIBRARY MAJOR VERSION")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
gold_schema = globals_dict['gold_schema']

# Versioning configuration from YAML
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

print(f"Catalog: {catalog}")
print(f"Gold Schema: {gold_schema}")
print(f"Versioning Config: {len(library_tables)} library table(s)")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Widget Parameters
dbutils.widgets.text("merge_strategy", "UNION_DEDUP", "Merge Strategy")
dbutils.widgets.text("library_type", "", "Library Type")

merge_strategy = dbutils.widgets.get("merge_strategy")
library_type_param = dbutils.widgets.get("library_type")

# Default to first library type from config if not specified
if library_type_param:
    library_type = library_type_param
elif library_tables:
    library_type = library_tables[0].get('library_type', 'transfer_variables')
else:
    library_type = 'transfer_variables'

print(f"\nMerge Strategy: {merge_strategy}")
print(f"Library Type: {library_type}")


In [ ]:
# Cell 4: Configure Table Names
library_table_map = {}
for lib_config in library_tables:
    lib_type = lib_config.get('library_type')
    lib_name = lib_config.get('name')
    lib_schema = lib_config.get('schema', gold_schema)
    if lib_type and lib_name:
        library_table_map[lib_type] = f"{catalog}.{lib_schema}.{lib_name}"

library_table = library_table_map.get(library_type, f"{catalog}.{gold_schema}.md_transfer_variables_library")
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

print(f"\nLibrary Table: {library_table}")
print(f"Registry Table: {registry_table}")


In [ ]:
# Cell 5: Query Active DTA Major Versions
print("\n" + "="*80)
print("Finding Active DTA Major Versions to Merge")
print("="*80)

# Query all active DTA_MAJOR versions for this library type
df_active_dtas = spark.sql(f"""
    SELECT version_tag, dta_id, record_count, created_ts
    FROM {registry_table}
    WHERE version_type = 'DTA_MAJOR'
      AND status = 'ACTIVE'
      AND library_type = '{library_type}'
    ORDER BY created_ts DESC
""")

dta_versions = [row.version_tag for row in df_active_dtas.collect()]

print(f"\nFound {len(dta_versions)} active DTA Major version(s):")
df_active_dtas.show(truncate=False)

if not dta_versions:
    raise ValueError(
        "No active DTA_MAJOR versions found to promote.\n"
        "Run job_cdm_dta_import_test first to create DTAs with DTA_MAJOR versions."
    )


In [ ]:
# Cell 6: Show Current Library Version
print("\n" + "="*80)
print("Current Library Major Version")
print("="*80)

try:
    current_lib = get_current_library_version(spark, library_table)
    current_count = current_lib.count()
    print(f"\nCurrent library has {current_count} records")
    current_lib.select("library_version", "is_current").limit(1).show()
except Exception as e:
    print(f"No current library version found (first promotion): {e}")


In [ ]:
# Cell 7: Execute Merge to Library Major
print("\n" + "="*80)
print(f"Merging {len(dta_versions)} DTA(s) -> Library Major")
print("="*80)

new_version = merge_dtas_to_library_major(
    spark=spark,
    library_table=library_table,
    registry_table=registry_table,
    dta_versions=dta_versions,
    library_type=library_type,
    merge_strategy=merge_strategy,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id
)

print(f"\n✅ New Library Major Version: {new_version}")


In [ ]:
# Cell 8: Set Task Values and Summary
dbutils.jobs.taskValues.set(key="new_library_version", value=new_version)
dbutils.jobs.taskValues.set(key="merged_from", value=json.dumps(dta_versions))
dbutils.jobs.taskValues.set(key="merge_strategy", value=merge_strategy)

print("\n" + "="*80)
print("Library Major Version History")
print("="*80)

all_versions = get_all_versions(spark, registry_table, library_type)
all_versions.filter("version_type = 'LIBRARY_MAJOR'").select(
    "version_tag", "status", "record_count", "created_ts"
).orderBy("created_ts", ascending=False).show(truncate=False)

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"New Library Version: {new_version}")
print(f"Merged from {len(dta_versions)} DTA(s):")
for v in dta_versions:
    print(f"  - {v}")
print(f"Strategy: {merge_strategy}")
